# Abstract Algebras

## References

* [Group Explorer](https://nathancarter.github.io/group-explorer/index.html) -- Visualization software for the abstract algebra classroom
* [Groupprops, The Group Properties Wiki (beta)](https://groupprops.subwiki.org/wiki/Main_Page)
* [Klein four-group, V4](https://en.wikipedia.org/wiki/Klein_four-group)
* [Cyclic group](https://en.wikipedia.org/wiki/Cyclic_group)

## Algebra Definition

In [94]:
def swap_list_items(lst, item1, item2):
    a, b = lst.index(item1), lst.index(item2)
    lst[b], lst[a] = lst[a], lst[b]
    return None

class Algebra:
    '''An abstract algebra with a finite number of elements and a multiplication table.'''
    
    def __init__(self, name, description, element_names, mult_table):
        self.name = name
        self.desc = description
        self.elements = element_names
        self.table = mult_table
        # For efficiency, calculate the headers up front
        self.col_header = self.table[0]
        self.row_header = [row[0] for row in self.table]
        
    def __str__(self):
        return f"<{self.__class__.__name__}: {self.name}, {self.desc}>"
    
    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}', '{self.desc}', {self.elements}, {self.table})"
    
    def mult_table_with_names(self):
        return [[self.elements[x] for x in row] for row in self.table]
        
    def mult(self, r, c):
        '''Return the product of elements, r & c.
        The inputs, r & c, can be numbers or strings, but if either
        input is a number, then a number will be returned, otherwise
        the product's element name (a string) will be returned.'''

        # Table lookup requires numbers
        r_ = r; c_ = c
        str_result = False
        if type(r) == str:
            r_ = self.elements.index(r)
            str_result = True
        if type(c) == str:
            c_ = self.elements.index(c)
            str_result = True
        
        # Lookup the product based on the row & column indices
        row_index = self.row_header.index(r_)
        col_index = self.col_header.index(c_)
        product = self.table[row_index][col_index]
        
        # If either input value was a string, then return a string,
        # otherwise return a number
        if str_result:
            return self.elements[product]
        else:
            return product
    
    def switch(self, elem1, elem2):
        lst = self.elements
        a, b = lst.index(elem1), lst.index(elem2)
        lst[b], lst[a] = lst[a], lst[b]
        tbl = self.table
        tbl[b], tbl[a] = tbl[a], tbl[b]
        return None
    
    def elements(self):
        return self.table[0]

## Group Definition

In [95]:
class Group(Algebra):
    pass

### Klein four-group, V4

In [96]:
v4 = Group('V4',
           'Klein four-group',
           ['e',  'h',  'v', ' hv'],
           [[0, 1, 2, 3],
            [1, 0, 3, 2],
            [2, 3, 0, 1],
            [3, 2, 1, 0]]
          )

In [97]:
v4.mult_table_with_names()

[['e', 'h', 'v', ' hv'],
 ['h', 'e', ' hv', 'v'],
 ['v', ' hv', 'e', 'h'],
 [' hv', 'v', 'h', 'e']]

In [98]:
v4.mult('h','v')

' hv'

In [99]:
print(v4)

<Group: V4, Klein four-group>


### Cyclic group of order 4

In [100]:
z4 = Group('Z4',
           'Cyclic group of order 4',
           ['e', 'a', 'a^2', 'a^3'],
           [[0, 1, 2, 3],
            [1, 2, 3, 0],
            [2, 3, 0, 1],
            [3, 0, 1, 2]]
          )

In [101]:
z4.mult_table_with_names()

[['e', 'a', 'a^2', 'a^3'],
 ['a', 'a^2', 'a^3', 'e'],
 ['a^2', 'a^3', 'e', 'a'],
 ['a^3', 'e', 'a', 'a^2']]

In [102]:
print(z4)

<Group: Z4, Cyclic group of order 4>


In [103]:
z4.mult(2,2)

0

In [104]:
z4.mult('a^2', 'a^2')

'e'

### Symmetric group on 3 letters

In [105]:
s3 = Group('S3',
           'Symmetric group on 3 letters',
           ['e', 'r', 'r^2', 'f', 'fr', 'rf'],
           [[0, 1, 2, 3, 4, 5],
            [1, 2, 0, 5, 3, 4],
            [2, 0, 1, 4, 5, 3],
            [3, 4, 5, 0, 1, 2],
            [4, 5, 3, 2, 0, 1],
            [5, 3, 4, 1, 2, 0]]
          )

In [106]:
s3.mult_table_with_names()

[['e', 'r', 'r^2', 'f', 'fr', 'rf'],
 ['r', 'r^2', 'e', 'rf', 'f', 'fr'],
 ['r^2', 'e', 'r', 'fr', 'rf', 'f'],
 ['f', 'fr', 'rf', 'e', 'r', 'r^2'],
 ['fr', 'rf', 'f', 'r^2', 'e', 'r'],
 ['rf', 'f', 'fr', 'r', 'r^2', 'e']]

In [107]:
print(s3)

<Group: S3, Symmetric group on 3 letters>


In [108]:
s3.mult('fr', 'r^2')

'f'

In [109]:
s3

Group('S3', 'Symmetric group on 3 letters', ['e', 'r', 'r^2', 'f', 'fr', 'rf'], [[0, 1, 2, 3, 4, 5], [1, 2, 0, 5, 3, 4], [2, 0, 1, 4, 5, 3], [3, 4, 5, 0, 1, 2], [4, 5, 3, 2, 0, 1], [5, 3, 4, 1, 2, 0]])

In [120]:
# This is also a version of the S3 group as specified at the Groupprops wiki (see reference, above).

s3x = Group('S3X',
           'Another version of the symmetric group on 3 letters',
           ['()', '(1,2)', '(2,3)', '(1,3)', '(1,2,3)', '(1,3,2)'],
           [[0, 1, 2, 3, 4, 5],
            [1, 0, 4, 5, 2, 3],
            [2, 5, 0, 4, 3, 1],
            [3, 4, 5, 0, 1, 2],
            [4, 3, 1, 2, 5, 0],
            [5, 2, 3, 1, 0, 4]]
          )

In [121]:
s3x.mult_table_with_names()

[['()', '(1,2)', '(2,3)', '(1,3)', '(1,2,3)', '(1,3,2)'],
 ['(1,2)', '()', '(1,2,3)', '(1,3,2)', '(2,3)', '(1,3)'],
 ['(2,3)', '(1,3,2)', '()', '(1,2,3)', '(1,3)', '(1,2)'],
 ['(1,3)', '(1,2,3)', '(1,3,2)', '()', '(1,2)', '(2,3)'],
 ['(1,2,3)', '(1,3)', '(1,2)', '(2,3)', '(1,3,2)', '()'],
 ['(1,3,2)', '(2,3)', '(1,3)', '(1,2)', '()', '(1,2,3)']]

In [122]:
s3x.elements

['()', '(1,2)', '(2,3)', '(1,3)', '(1,2,3)', '(1,3,2)']

In [123]:
s3x.switch('(1,2,3)', '(1,3,2)')

In [124]:
s3x.elements

['()', '(1,2)', '(2,3)', '(1,3)', '(1,3,2)', '(1,2,3)']

In [125]:
s3x.mult_table_with_names()

[['()', '(1,2)', '(2,3)', '(1,3)', '(1,3,2)', '(1,2,3)'],
 ['(1,2)', '()', '(1,3,2)', '(1,2,3)', '(2,3)', '(1,3)'],
 ['(2,3)', '(1,2,3)', '()', '(1,3,2)', '(1,3)', '(1,2)'],
 ['(1,3)', '(1,3,2)', '(1,2,3)', '()', '(1,2)', '(2,3)'],
 ['(1,2,3)', '(2,3)', '(1,3)', '(1,2)', '()', '(1,3,2)'],
 ['(1,3,2)', '(1,3)', '(1,2)', '(2,3)', '(1,2,3)', '()']]